In [18]:
pip install pyarrow

     |████████████████████████████████| 19.1 MB 8.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
import boto3
import requests
import base64
import json
import pandas as pd
from datetime import datetime
import jsonpath
import pymysql

In [ ]:
def get_headers(client_id, client_secret):
    endpoint = 'https://accounts.spotify.com/api/token'
    encoded = base64.b64encode((client_id+':'+client_secret).encode('utf-8')).decode('ascii')
    headers = {'Authorization': 'Basic {auth}'.format(auth=encoded)}
    payload = {'grant_type':'client_credentials'}
    try:
        r = requests.post(endpoint, data=payload, headers=headers)
    except:
        print('인증오류가 발생되었습니다!')
        sys.exit(1)
    if r.status_code!=200:
        if r.status_code==401:
            print('Client_id와 Secret이 잘못되었습니다!')
            sys.exit(1)
        else:
            print('인증오류가 발생되었습니다!:', r.status_code)
            sys.exit(1)
    acess_token = json.loads(r.text).get('access_token')
    headers = {'Authorization': 'Bearer {auth}'.format(auth=acess_token)}
    return headers

In [ ]:
def id_getter(que):
    endpoint = 'https://api.spotify.com/v1/search'
    headers = get_headers(client_id, client_secret)
    params = {'q': que,
              'type':'artist',
              'market':'KR',
              'limit':1}
    try:
        r = requests.get(endpoint, params=params, headers=headers)
    except:
        print('ID획득에 오류가 발생되었습니다!')
        sys.exit(1)
    if r.status_code!=200:
        if r.status_code==401:
            headers = get_headers(client_id, client_secret)
            r = requests.get(endpoint, params=params, headers=headers)
        elif r.status_code==429:
            retry_time = json.loads(r.headers)['Retry-After']
            time.sleep(int(retry_time) + 1)
            r = requests.get(endpoint, params=params, headers=headers)
        else:
            print('ID획득에 오류가 발생되었습니다!',r.status_code)
            sys.exit(1)
    try:
        my_id = json.loads(r.text).get('artists').get('items')[0].get('id')
    except:
        print('ID가 존재하지 않습니다.', que)
        return None
    return my_id

In [ ]:
def artist_info(client_id, client_secret, que):
    seq = 50
    idsets = [que[i:i+seq] for i in range(0, len(que), seq)]
    a_info_list = []
    
    for artist_ids in idsets:
        endpoint = 'https://api.spotify.com/v1/artists?ids={a_id}'.format(a_id=','.join(artist_ids))
        headers = get_headers(client_id, client_secret)
        try:
            r = requests.get(endpoint, headers=headers)
        except:
            print('Artist정보 획득에 오류가 발생되었습니다.')
            sys.exit(1)

        if r.status_code!=200:
            if r.status_code==401:
                headers = get_headers(client_id, client_secret)
                r = requests.get(endpoint, params=params, headers=headers)
            elif r.status_code==429:
                retry_time = json.loads(r.headers)['Retry-After']
                time.sleep(int(retry_time) + 1)
                r = requests.get(endpoint, params=params, headers=headers)
            else:
                print('Artist정보 획득에 오류가 발생되었습니다!',r.status_code)
                sys.exit(1)
        
        full_list = json.loads(r.text)
        for raw in full_list['artists']:
            artist_dict = {}
            artist_genre = {}
            try:
                artist_dict['artist_id'] = raw['id']
                artist_genre['artist_id'] = raw['id']
            except:
                print('Artist ID가 없습니다!')
                sys.exit(1)

            try:
                artist_dict['name'] = raw.get('name','UNK')
                artist_dict['followers'] = raw.get('followers', 0).get('total',0)
                artist_dict['popularity'] = raw.get('popularity', 0)
                artist_dict['url'] = raw.get('external_urls', 'UNK').get('spotify','UNK')
                artist_genre['genres'] = raw.get('genres')
            except:
                print('Artist 정보획득에 실패하였습니다.')
                sys.exit(1)
            try:
                artist_dict['image_url'] = raw.get('images', 'UNK')[0].get('url','UNK')
            except:
                artist_dict['image_url'] = 'UNK'                
            a_info_list.append((artist_dict, artist_genre))
    return a_info_list

In [ ]:
def insert_row(cursor, data, table):
    col = ', '.join(data)
    place_holders = ', '.join(['%s']*len(data))
    key_holders = ', '.join([k+'=%s' for k in data.keys()])
    que = 'INSERT INTO {} ({}) VALUES ({}) ON DUPLICATE KEY UPDATE {}'.format(table, col, place_holders, key_holders)
    cursor.execute(que, list(data.values())*2)

In [ ]:
with open('artist_list_2021.csv', 'r', encoding='euc-kr') as f:
    data = f.read()
    data = data.split('\n')
    data = data[:500]
    
client_id = '9e22029795364c83be0d5c1c8ea99657'
client_secret = ''
my_ids = [id_getter(d) for d in data]

a_info = artist_info(client_id, client_secret, my_ids)

In [9]:
host = "hyundb.cqgrmpcsglic.ap-northeast-2.rds.amazonaws.com"
port = 3306
username = 'hyunjilee'
database = 'production'
pw = ''
conn = pymysql.connect(host=host, user=username, passwd=pw, db=database,
                           port=port, use_unicode=True, charset='utf8')
cursor = conn.cursor()

In [10]:
for d in a_info:
    insert_row(cursor, d[0], 'artists')
    if len(d[1]['genres'])==0:
        continue
    for g in d[1]['genres']:
        g_dict = {'artist_id':d[1]['artist_id'], 'genre':g}
        insert_row(cursor, g_dict, 'artist_genres')
    conn.commit()
cursor.close()
conn.close()

In [ ]:
if __name__=='__main__':
    main()

In [2]:
try:
    host = "hyundb.cqgrmpcsglic.ap-northeast-2.rds.amazonaws.com"
    port = 3306
    username = 'hyunjilee'
    database = 'production'
    pw = 'dlguswlek0304!'
    conn = pymysql.connect(host=host, user=username, passwd=pw, db=database,
                               port=port, use_unicode=True, charset='utf8')
    cursor = conn.cursor()
except : 
    print('RDS접속오류!')
    sys.exit(1)

cursor.execute('SELECT * FROM artists')
data = pd.DataFrame(cursor.fetchall())

In [3]:
data

,0,1,2,3,4,5
0,00FQb4jTyendYWaN8pK0wa,Lana Del Rey,14434807,88,https://open.spotify.com/artist/00FQb4jTyendYW...,https://i.scdn.co/image/ab6761610000e5ebc59036...
1,01C9OoXDvCKkGcf735Tcfo,Serge Gainsbourg,425124,61,https://open.spotify.com/artist/01C9OoXDvCKkGc...,https://i.scdn.co/image/ab6761610000e5ebac7fa5...
2,01nsKMfCWABk4pZsOUM3YD,Bj√∂rk,8,0,https://open.spotify.com/artist/01nsKMfCWABk4p...,UNK
3,02frazNrWgZCxUEf4UTfHt,Van Der Graaf Generator,103119,40,https://open.spotify.com/artist/02frazNrWgZCxU...,https://i.scdn.co/image/ab67616d0000b2736931cd...
4,02rd0anEWfMtF7iMku9uor,Reba McEntire,1498231,66,https://open.spotify.com/artist/02rd0anEWfMtF7...,https://i.scdn.co/image/ab6761610000e5eb3cbed3...
...,...,...,...,...,...,...
497,7xkAwz0bQTGDSbkofyQt3U,Belinda Carlisle,599416,65,https://open.spotify.com/artist/7xkAwz0bQTGDSb...,https://i.scdn.co/image/ab6761610000e5eb70c572...
498,7xTKLpo7UCzXSnlH7fOIoM,Redman,861711,69,https://open.spotify.com/artist/7xTKLpo7UCzXSn...,https://i.scdn.co/image/0b05a9d1bfb856e3c55ae6...
499,7y70dch6JuuuNnwlsOQvwW,Nitty Gritty Dirt Band,140788,63,https://open.spotify.com/artist/7y70dch6JuuuNn...,https://i.scdn.co/image/ab6761610000e5ebeeb8bb...
500,7zfhej6FnVXN9LIXs6dcoK,The Tubes,67834,48,https://open.spotify.com/artist/7zfhej6FnVXN9L...,https://i.scdn.co/image/ab6761610000e5ebc645d7...


In [4]:
data = data.rename(columns={0:'artist_id', 1: 'name', 2: 'followers',
                           3: 'popularity', 4: 'url', 5:'image_url'})

In [7]:
data.to_parquet('artists.parquet', engine='pyarrow', 
                compression='snappy')
dt = datetime.utcnow().strftime('%Y-%m-%d')  # 현재시간 
s3 = boto3.resource('s3', region_name='ap-northeast-2')
ob = s3.Object('hyun-artist', 'artists/dt={}/artists.parquet'.format(dt))
data = open('artists.parquet', 'rb')
ob.put(Body=data)

{'ResponseMetadata': {'RequestId': 'W4EQ5QKAVEPD181X',
  'HostId': 'aY0jap4dlCc7sXQPcryMewYt1XBs0irpn+z9J43MyZZRlTXDtxwyez0nfyk4G7c7MCM2vUybiak=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'aY0jap4dlCc7sXQPcryMewYt1XBs0irpn+z9J43MyZZRlTXDtxwyez0nfyk4G7c7MCM2vUybiak=',
   'x-amz-request-id': 'W4EQ5QKAVEPD181X',
   'date': 'Sun, 14 Nov 2021 13:42:22 GMT',
   'etag': '"31740418436f58b21fb772fecd3725fc"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"31740418436f58b21fb772fecd3725fc"'}

In [12]:
try:
    host = "hyundb.cqgrmpcsglic.ap-northeast-2.rds.amazonaws.com"
    port = 3306
    username = 'hyunjilee'
    database = 'production'
    pw = 'dlguswlek0304!'
    conn = pymysql.connect(host=host, user=username, passwd=pw, db=database,
                               port=port, use_unicode=True, charset='utf8')
    cursor = conn.cursor()
except :
    print('RDS접속오류!')
    sys.exit(1)

cursor.execute('SELECT * FROM artist_genres')
data = pd.DataFrame(cursor.fetchall())

In [14]:
data = data.rename(columns={0:'artist_id', 1: 'genre'})

In [15]:
data.to_parquet('artist_genres.parquet', engine='pyarrow', 
                compression='snappy')
dt = datetime.utcnow().strftime('%Y-%m-%d')  # 현재시간 
s3 = boto3.resource('s3', region_name='ap-northeast-2')
ob = s3.Object('hyun-artist', 'artist_genres/dt={}/artist_genres.parquet'.format(dt))
data = open('artist_genres.parquet', 'rb')
ob.put(Body=data)

{'ResponseMetadata': {'RequestId': 'TKRJWMR33ZSWNGVH',
  'HostId': 'Zgy1OgAdEv4ps4FUMk8UixVOZZnrCnjtJUqi29sSldQal4mJsrPdybbZVXavdQ658fyhlEzAH6E=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Zgy1OgAdEv4ps4FUMk8UixVOZZnrCnjtJUqi29sSldQal4mJsrPdybbZVXavdQ658fyhlEzAH6E=',
   'x-amz-request-id': 'TKRJWMR33ZSWNGVH',
   'date': 'Sun, 14 Nov 2021 13:47:43 GMT',
   'etag': '"7bb87a7eca7ca539d74a7a0767b64b22"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"7bb87a7eca7ca539d74a7a0767b64b22"'}